In [1]:
%pip show langchain #Version: 0.1.14

Name: langchainNote: you may need to restart the kernel to use updated packages.

Version: 0.1.14
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\Users\Hani\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: aiohttp, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Imports and Environment

In [1]:
import os

In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader # type: ignore
from langchain.text_splitter import RecursiveCharacterTextSplitter # type: ignore
from langchain.embeddings import OpenAIEmbeddings # type: ignore
from langchain.llms import OpenAI # type: ignore
from langchain.vectorstores import Pinecone# type: ignore
from langchain.chains import RetrievalQA# type: ignore
from langchain.prompts import PromptTemplate# type: ignore
from langchain.chains.summarize import load_summarize_chain# type: ignore
from langchain_community.chat_models import ChatOpenAI# type: ignore

# Load and Chunks Doucment

In [3]:
loader = PyPDFDirectoryLoader("pdf")

In [4]:
data = loader.load()

In [5]:
data[:5]

[Document(page_content='Download free eBooks of classic literature, books and \nnovels at Planet eBook. Subscribe to our free eBooks blog \nand email newsletter.Crime and Punishment\nBy Fyodor Dostoevsky\n', metadata={'source': 'pdf\\crime-and-punishment.pdf', 'page': 0}),
 Document(page_content='Crime and Punishment \x18Translator’s Preface\nA few words about Dostoevsky himself may help the Eng -\nlish reader to understand his work.\nDostoevsky was the son of a doctor. His parents were \nvery hard- working and deeply religious people, but so poor \nthat they lived with their five children in only two rooms. \nThe father and mother spent their evenings in reading aloud \nto their children, generally from books of a serious charac -\nter.\nThough always sickly and delicate Dostoevsky came out \nthird in the final examination of the Petersburg school of \nEngineering. There he had already begun his first work, \n‘Poor Folk.’\nThis story was published by the poet Nekrassov in his \nreview

In [6]:
print(f"You have {len(data)} pages") 

You have 767 pages


# Preprocessing

In [7]:
from pprint import pprint
print(data[6].page_content)

 Free eBooks at Planet eBook.comChapter I
On an exceptionally hot evening early in July a young 
man came out of the garret in which he lodged in S. 
Place and walked slowly, as though in hesitation, towards 
K. bridge.
He had successfully avoided meeting his landlady on the 
staircase. His garret was under the roof of a high, five-sto -
ried house and was more like a cupboard than a room. The 
landlady who provided him with garret, dinners, and at -
tendance, lived on the floor below, and every time he went 
out he was obliged to pass her kitchen, the door of which 
invariably stood open. And each time he passed, the young 
man had a sick, frightened feeling, which made him scowl 
and feel ashamed. He was hopelessly in debt to his landlady, 
and was afraid of meeting her.
This was not because he was cowardly and abject, quite 
the contrary; but for some time past he had been in an over -
strained irritable condition, verging on hypochondria. He 
had become so completely absorbed in h

`Remove opening part`

In [8]:
data_remove_authorsnote = data[5:]

In [9]:
data_remove_authorsnote[1].page_content

'\x18 Free eBooks at Planet eBook.comChapter I\nOn an exceptionally hot evening early in July a young \nman came out of the garret in which he lodged in S. \nPlace and walked slowly, as though in hesitation, towards \nK. bridge.\nHe had successfully avoided meeting his landlady on the \nstaircase. His garret was under the roof of a high, five-sto -\nried house and was more like a cupboard than a room. The \nlandlady who provided him with garret, dinners, and at -\ntendance, lived on the floor below, and every time he went \nout he was obliged to pass her kitchen, the door of which \ninvariably stood open. And each time he passed, the young \nman had a sick, frightened feeling, which made him scowl \nand feel ashamed. He was hopelessly in debt to his landlady, \nand was afraid of meeting her.\nThis was not because he was cowardly and abject, quite \nthe contrary; but for some time past he had been in an over -\nstrained irritable condition, verging on hypochondria. He \nhad become so co

In [10]:
for string in data_remove_authorsnote:
    string.page_content = string.page_content.replace("Free eBooks at Planet eBook.com", "")


**Crime and Punishment /x18**

In [11]:
print(data_remove_authorsnote[2].page_content)

Crime and Punishment stopped on the stairs, to be forced to listen to her trivial, ir -
relevant gossip, to pestering demands for payment, threats 
and complaints, and to rack his brains for excuses, to pre -
varicate, to lie—no, rather than that, he would creep down 
the stairs like a cat and slip out unseen.
This evening, however, on coming out into the street, he 
became acutely aware of his fears.
‘I want to attempt a thing like that  and am frightened 
by these trifles,’ he thought, with an odd smile. ‘Hm … yes, 
all is in a man’s hands and he lets it all slip from cowardice, 
that’s an axiom. It would be interesting to know what it is 
men are most afraid of. Taking a new step, uttering a new 
word is what they fear most…. But I am talking too much. 
It’s because I chatter that I do nothing. Or perhaps it is that 
I chatter because I do nothing. I’ve learned to chatter this 
last month, lying for days together in my den thinking … 
of Jack the Giant-killer. Why am I going there 

**Crime and Punishment (without /x18)**
- Integers are a non trivial case, cannot just remove them using regex

In [12]:
print(data_remove_authorsnote[4].page_content)

Crime and Punishment 10heavy dray horse, suddenly shouted at him as he drove past: 
‘Hey there, German hatter’ bawling at the top of his voice 
and pointing at him—the young man stopped suddenly 
and clutched tremulously at his hat. It was a tall round hat 
from Zimmerman’s, but completely worn out, rusty with 
age, all torn and bespattered, brimless and bent on one side 
in a most unseemly fashion. Not shame, however, but quite 
another feeling akin to terror had overtaken him.
‘I knew it,’ he muttered in confusion, ‘I thought so! That’s 
the worst of all! Why, a stupid thing like this, the most trivial 
detail might spoil the whole plan. Yes, my hat is too notice -
able…. It looks absurd and that makes it noticeable…. With 
my rags I ought to wear a cap, any sort of old pancake, but 
not this grotesque thing. Nobody wears such a hat, it would 
be noticed a mile off, it would be remembered…. What mat -
ters is that people would remember it, and that would give 
them a clue. For this b

In [13]:
for string in data_remove_authorsnote:
    string.page_content = string.page_content.replace("Crime and Punishment \x18", "")
    if string.page_content.startswith("Crime and Punishment"):
        string.page_content = string.page_content[len("Crime and Punishment"):]  # Remove the prefix

In [14]:
print(data_remove_authorsnote[1].page_content)

 Chapter I
On an exceptionally hot evening early in July a young 
man came out of the garret in which he lodged in S. 
Place and walked slowly, as though in hesitation, towards 
K. bridge.
He had successfully avoided meeting his landlady on the 
staircase. His garret was under the roof of a high, five-sto -
ried house and was more like a cupboard than a room. The 
landlady who provided him with garret, dinners, and at -
tendance, lived on the floor below, and every time he went 
out he was obliged to pass her kitchen, the door of which 
invariably stood open. And each time he passed, the young 
man had a sick, frightened feeling, which made him scowl 
and feel ashamed. He was hopelessly in debt to his landlady, 
and was afraid of meeting her.
This was not because he was cowardly and abject, quite 
the contrary; but for some time past he had been in an over -
strained irritable condition, verging on hypochondria. He 
had become so completely absorbed in himself, and iso -
lated from hi

**Larger chunk size for tasks since summarization require a more holistic view of the text.**

In [15]:
print(len(data_remove_authorsnote[2].page_content), "characters per page")

1669 characters per page


In [16]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 700, chunk_overlap=20, 
#                                                separators=[". ",", ","\n",""] ) 
#split into equal chunks

# default separator list of ["\n\n", "\n", " ", ""] 
#can cause words to be split between chunks

In [17]:
# text_chunks = text_splitter.split_documents(data_remove_authorsnote[:3])

In [18]:
# text_chunks

# Experimenting Approaches for summarization

**We will use only first two chapters for experimentation**

In [38]:
data_remove_authorsnote[38].page_content

'\x18Chapter III\nHe waked up late next day after a broken sleep. But his \nsleep had not refreshed him; he waked up bilious, ir -\nritable, ill-tempered, and looked with hatred at his room. \nIt was a tiny cupboard of a room about six paces in length. \nIt had a poverty-stricken appearance with its dusty yellow \npaper peeling off the walls, and it was so low-pitched that \na man of more than average height was ill at ease in it and \nfelt every moment that he would knock his head against the \nceiling. The furniture was in keeping with the room: there \nwere three old chairs, rather rickety; a painted table in the \ncorner on which lay a few manuscripts and books; the dust \nthat lay thick upon them showed that they had been long \nuntouched. A big clumsy sofa occupied almost the whole \nof one wall and half the floor space of the room; it was once \ncovered with chintz, but was now in rags and served Ras -\nkolnikov as a bed. Often he went to sleep on it, as he was, \nwithout undres

**Notes**
- stuff
- map reduce
- refine
- tree summrisation

In [42]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [57]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106", openai_api_key=OPENAI_API_KEY)

In [58]:
data_sample= data_remove_authorsnote[:38]
data_sample[-1]

Document(page_content='\x18\x18 there! And they’re making the most of it! Yes, they are mak -\ning the most of it! They’ve wept over it and grown used to it. \nMan grows used to everything, the scoundrel!’\nHe sank into thought.\n‘And what if I am wrong,’ he cried suddenly after a mo -\nment’s thought. ‘What if man is not really a scoundrel, man \nin general, I mean, the whole race of mankind—then all the \nrest is prejudice, simply artificial terrors and there are no \nbarriers and it’s all as it should be.’', metadata={'source': 'pdf\\crime-and-punishment.pdf', 'page': 42})

## Stuff

In [93]:
from langchain.chains.llm import LLMChain

prompt_template = """As an expert summarizer, your task is to extract key information from the text of Crime and Punishment,
 and then apply the provided instructions.

1. Prioritize the identification of the Part and Chapter provided within the text.
2. Skim the document to understand its overall theme and intent. 
3. Extract the core essence and primary themes of the document, ensuring to represent the author's primary intent accurately.
4. Condense any significant points, arguments, or charatcer's thoughts into a clear and succinct overview.
5. For any events mentioned in the text, provide the characters involved, date, time and place of the event ensuring that the essence of the event is captured succinctly.
6. Present a compact summary that encapsulates:
- Condensed significant points, arguments, or charatcer's thoughts into a clear and succinct overview.
- Any events that take place and their outcome.


Your response should be a succinct paragraph, not exceeding 200 words, encapsulating the key information of the text.
 Avoid including any information outside the scope of the text's summary. ALWAYS give a summary of the given text.

Text to be summarized is enclosed within ``` ``` markers:

```{text}```

CONCISE SUMMARY:"""
prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [94]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

In [95]:
data_sample[:13]

[Document(page_content='Part I', metadata={'source': 'pdf\\crime-and-punishment.pdf', 'page': 5}),
 Document(page_content='\x18 Chapter I\nOn an exceptionally hot evening early in July a young \nman came out of the garret in which he lodged in S. \nPlace and walked slowly, as though in hesitation, towards \nK. bridge.\nHe had successfully avoided meeting his landlady on the \nstaircase. His garret was under the roof of a high, five-sto -\nried house and was more like a cupboard than a room. The \nlandlady who provided him with garret, dinners, and at -\ntendance, lived on the floor below, and every time he went \nout he was obliged to pass her kitchen, the door of which \ninvariably stood open. And each time he passed, the young \nman had a sick, frightened feeling, which made him scowl \nand feel ashamed. He was hopelessly in debt to his landlady, \nand was afraid of meeting her.\nThis was not because he was cowardly and abject, quite \nthe contrary; but for some time past he had been

`uses 15k tokens for first two chapters`

In [96]:
stuff_summary = stuff_chain.run(data_sample)

In [97]:
stuff_summary

"The text follows a young man, Raskolnikov, as he walks through the streets of Petersburg, struggling with poverty and inner turmoil. He is deeply in debt to his landlady and is avoiding her, feeling isolated and irritable. He contemplates taking a new step but is plagued by fear and self-doubt. He encounters a drunken man, Marmeladov, who shares his tragic story of poverty and family struggles. Marmeladov's wife, Katerina Ivanovna, is in despair over their dire situation. Raskolnikov witnesses their distress and leaves money for them before walking away, reflecting on the struggles of those around him."

In [98]:
num_tokens = llm.get_num_tokens(stuff_summary)
num_tokens

140

In [99]:
print(len(stuff_summary), "characters")
##roughly 2000 characters per page are ideal

609 characters


In [100]:

# Specify the file path where you want to save the text file
file_path = "stuff_summary_2.txt"

# Open the file in write mode and save the string
with open(file_path, "w") as file:
    file.write(stuff_summary)

print(f"String saved to {file_path}")

String saved to stuff_summary_2.txt


`The stuffing method is a way to summarize text by feeding the entire document to LLM in a single call. If the document is longer than the context length, the stuffing method will not work. Also the stuffing method is not suitable for summarizing large documents, as it can be slow and may not produce a good summary. Not suitable for our case`

## Map Reduce

The MapReduce method implements a multi-stage summarization. It is a technique for summarizing large pieces of text by first summarizing smaller chunks of text and then combining those summaries into a single summary.

In [174]:
map_prompt_template = """
                      Write a concise summary of the following: 
                       - if any events that took place, 
                       - thoughts of the characters mentioned,
                       - and retains essence of the text
                      {text}
                      SUMMARY: 
                      """
map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """As an expert summarizer, your task is to extract key information from the text of Crime and Punishment,
 and then apply the provided instructions.

1. Prioritize the identification of the Part and Chapter provided within the text.
2. Skim the document to understand its overall theme and intent. 
3. Extract the core essence and primary themes of the document, ensuring to represent the author's primary intent accurately.
4. Condense any significant points, arguments, or charatcer's thoughts into a clear and succinct overview.
5. For any events mentioned in the text, provide the characters involved, date, time and place of the event ensuring that the essence of the event is captured succinctly.
6. Present a compact summary of the story that encapsulates:
- Condensed significant points, arguments, or charatcer's thoughts into a clear and succinct overview.
- Any events that take place, in respective order and their outcome.


Your response should be a succinct paragraph, not exceeding 200 words, encapsulating the key information of the text.
Avoid including any information outside the scope of the text's summary. ALWAYS give a summary of the given text.

Text to be summarized is enclosed within ``` ``` markers:

```{text}```

CONCISE SUMMARY:"""

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

In [175]:
map_reduce_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

In [176]:
map_reduce_outputs = map_reduce_chain({"input_documents": data_sample})

In [179]:
map_reduce_outputs["output_text"]

"The text focuses on the internal struggles and external impact on the characters in Part I, with Raskolnikov experiencing poverty and anxiety. In Chapter I, he has a daring dream and visits an old woman, highlighting desperation and poverty. In Chapter II, he encounters Marmeladov in a filthy tavern, reflecting his internal change. The text captures the characters' hardships, desperation, and hope for redemption, as well as Raskolnikov's internal turmoil and the impact of poverty on their lives. The chaotic scene in a room with Katerina Ivanovna struggling and accusing her husband of being a criminal is also described, with Raskolnikov encountering the situation and being accused as well. The characters reflect on the resilience of mankind and the ability to adapt to difficult circumstances, questioning the negative view of humanity and contemplating the potential for change in perspective."

In [180]:
# Specify the file path where you want to save the text file
file_path = "map_reduce_4.txt"

# Open the file in write mode and save the string
with open(file_path, "w") as file:
    file.write(map_reduce_outputs["output_text"])

print(f"String saved to {file_path}")

String saved to map_reduce_4.txt


### Intermediate steps saved

In [107]:
from pathlib import Path
final_mp_data = []
for doc, out in zip(
    map_reduce_outputs["input_documents"], map_reduce_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = Path(doc.metadata["source"]).stem
    output["file_type"] = Path(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_mp_data.append(output)

In [109]:
import pandas as pd
pdf_mp_summary = pd.DataFrame.from_dict(final_mp_data)
pdf_mp_summary = pdf_mp_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the dataframe by filename and page_number
pdf_mp_summary.reset_index(inplace=True, drop=True)
pdf_mp_summary.head()

,file_name,file_type,page_number,chunks,concise_summary
0,crime-and-punishment,.pdf,5,Part I,"In this text, the main points revolve around t..."
1,crime-and-punishment,.pdf,6, Chapter I\nOn an exceptionally hot evening e...,"Summary:\nIn Chapter I, a young man living in ..."
2,crime-and-punishment,.pdf,7,"stopped on the stairs, to be forced to listen ...",The text describes a man feeling overwhelmed b...
3,crime-and-punishment,.pdf,8, An expression of the profoundest disgust gle...,The text describes a young man who is exceptio...
4,crime-and-punishment,.pdf,9,"10heavy dray horse, suddenly shouted at him a...",The text describes a young man who is suddenly...


In [112]:
index = 3
print("\n[Simple Summary]")
print(pdf_mp_summary["concise_summary"].iloc[index])
print("\n[Page number]")
print(pdf_mp_summary["page_number"].iloc[index])


[Simple Summary]
The text describes a young man who is exceptionally handsome but badly dressed, walking through the streets of Petersburg in a state of deep thought and physical weakness. He is so disgusted with his surroundings and his own circumstances that he does not care about his appearance, even though he is aware of his shabbiness. The area he is in is filled with a variety of people, so his appearance does not stand out. He also dislikes meeting acquaintances or former fellow students. The text conveys a sense of bitterness and contempt in the young man's heart.

[Page number]
8


In [164]:
# Save the summary to a text file
output_file_path = "map_reduce_1.txt"
with open(output_file_path, "w") as file:
    for index, row in pdf_mp_summary.iterrows():
        file.write(f"[Page Number]: {row['page_number']}\n")
        file.write(f"[Concise Summary]: {row['concise_summary']}\n\n")

print(f"Summary saved to {output_file_path}")

Summary saved to map_reduce_1.txt


### summary from intermediate steps

In [165]:
summary_string = "\n\n".join(pdf_mp_summary["concise_summary"])

In [120]:
mapr_summary = llm.invoke(f"""As an expert summarizer, your task is to extract key information from the text of Crime and Punishment,
 and then apply the provided instructions.

1. Prioritize the identification of the Part and Chapter provided within the text.
2. Skim the document to understand its overall theme and intent. 
3. Extract the core essence and primary themes of the document, ensuring to represent the author's primary intent accurately.
4. Condense any significant points, arguments, or charatcer's thoughts into a clear and succinct overview.
5. For any events mentioned in the text, provide the characters involved, date, time and place of the event ensuring that the essence of the event is captured succinctly.
6. Present a compact summary that encapsulates:
- Condensed significant points, arguments, or charatcer's thoughts into a clear and succinct overview.
- Any events that take place and their outcome.


Your response should be a succinct paragraph, not exceeding 200 words, encapsulating the key information of the text.
 Avoid including any information outside the scope of the text's summary. ALWAYS give a summary of the given text.

Text to be summarized is enclosed within ``` ``` markers:

```{summary_string}```

CONCISE SUMMARY:""")

In [126]:
mapr_summary.content

"The text focuses on the events and thoughts of the characters in Part I, introducing them and exploring their emotions. It describes a young man, Raskolnikov, feeling overwhelmed by poverty and isolation, and contemplating a new endeavor. He visits an old woman to pawn a watch, but she refuses to give him a fair price. Raskolnikov then demands something from her and listens to her movements to figure out where she keeps her keys. He becomes overwhelmed with fear and confusion, realizing that his hat could ruin his plans. The text also portrays the desperation and poverty of Marmeladov and his family, as well as the emotional turmoil and despair of the characters. The setting is described as dark and disorderly, with chaotic and tense atmospheres. The characters express feelings of worthlessness, hopelessness, and guilt, and struggle with poverty, alcoholism, and desperation. The text captures the internal struggles and moral ambiguity of the characters' actions and thoughts."

In [127]:
# Specify the file path where you want to save the text file
file_path = "map_reduce_2.txt"

# Open the file in write mode and save the string
with open(file_path, "w") as file:
    file.write(mapr_summary.content)

print(f"String saved to {file_path}")

String saved to map_reduce_2.txt


MapReduce requires multiple calls to the model and potentially losing context between pages.

## Refine

In [128]:
question_prompt_template = """Provide a summary of this chunk of text that includes:
                       - the main points and any important details, 
                       - if any events that took place, 
                       - thoughts of the characters mentioned,
                       - and retains essence of the text
                      {text}
                  SUMMARY:
                  """

question_prompt = PromptTemplate(
    template=question_prompt_template, input_variables=["text"]
)

refine_prompt_template = """
              As an expert summarizer, your task is to extract key information from the text of Crime and Punishment,
 and then apply the provided instructions.

1. Prioritize the identification of the Part and Chapter provided within the text.
2. Skim the text to understand its overall theme and intent. 
3. Extract the core essence and primary themes of the text, ensuring to represent the author's primary intent accurately.
4. Condense any significant points, arguments, or charatcer's thoughts into a clear and succinct overview.
5. For any events mentioned in the text, provide the characters involved, date, time and place of the event ensuring that the essence of the event is captured succinctly.
6. Present a compact summary that encapsulates:
- Condensed significant points, arguments, or charatcer's thoughts into a clear and succinct overview.
- Any events that take place and their outcome.


Your response should be a succinct paragraph, not exceeding 200 words, encapsulating the key information of the text.
 Avoid including any information outside the scope of the text's summary. ALWAYS give a summary of the given text.

Text to be summarized is enclosed within ``` ``` markers:

```{text}```

CONCISE SUMMARY:
              """

refine_prompt = PromptTemplate(
    template=refine_prompt_template, input_variables=["text"]
)

In [129]:
refine_chain = load_summarize_chain(
    llm,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
)

In [130]:
refine_outputs = refine_chain({"input_documents": data_sample})

In [168]:
refine_outputs["output_text"]

"The text explores the idea of man's adaptability to difficult circumstances and questions whether mankind is inherently corrupt. The protagonist reflects on the possibility that humanity is not inherently wicked, and that societal prejudices and fears create artificial barriers. The text highlights the theme of resilience and adaptation, as well as the philosophical contemplation of human nature. The protagonist's internal struggle with these ideas is evident, as he grapples with the concept of man's inherent nature. The text does not mention any specific events, characters, dates, or places."

In [131]:
from pathlib import Path
final_refine_data = []
for doc, out in zip(
    refine_outputs["input_documents"], refine_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = Path(doc.metadata["source"]).stem
    output["file_type"] = Path(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_refine_data.append(output)

In [132]:
pdf_refine_summary = pd.DataFrame.from_dict(final_refine_data)
pdf_refine_summary = pdf_mp_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the datafram by filename and page_number
pdf_refine_summary.reset_index(inplace=True, drop=True)
pdf_refine_summary.head()

,file_name,file_type,page_number,chunks,concise_summary
0,crime-and-punishment,.pdf,5,Part I,"In this text, the main points revolve around t..."
1,crime-and-punishment,.pdf,6, Chapter I\nOn an exceptionally hot evening e...,"Summary:\nIn Chapter I, a young man living in ..."
2,crime-and-punishment,.pdf,7,"stopped on the stairs, to be forced to listen ...",The text describes a man feeling overwhelmed b...
3,crime-and-punishment,.pdf,8, An expression of the profoundest disgust gle...,The text describes a young man who is exceptio...
4,crime-and-punishment,.pdf,9,"10heavy dray horse, suddenly shouted at him a...",The text describes a young man who is suddenly...


In [133]:
index = 3
print("\n\n [Simple Summary]")
print(pdf_refine_summary["concise_summary"].iloc[index])
print("\n\n [Page number]")
print(pdf_refine_summary["page_number"].iloc[index])




 [Simple Summary]
The text describes a young man who is exceptionally handsome but badly dressed, walking through the streets of Petersburg in a state of deep thought and physical weakness. He is so disgusted with his surroundings and his own circumstances that he does not care about his appearance, even though he is aware of his shabbiness. The area he is in is filled with a variety of people, so his appearance does not stand out. He also dislikes meeting acquaintances or former fellow students. The text conveys a sense of bitterness and contempt in the young man's heart.


 [Page number]
8


In [134]:
# Save the summary to a text file
output_file_path = "refine_1.txt"
with open(output_file_path, "w") as file:
    for index, row in pdf_refine_summary.iterrows():
        file.write(f"[Page Number]: {row['page_number']}\n")
        file.write(f"[Concise Summary]: {row['concise_summary']}\n\n")

print(f"Summary saved to {output_file_path}")

Summary saved to refine_1.txt


In [137]:
summary_string_refine = "\n\n".join(pdf_refine_summary["concise_summary"])

In [138]:
refine_summary = llm.invoke(f"""As an expert summarizer, your task is to extract key information from the text of Crime and Punishment,
 and then apply the provided instructions.

1. Prioritize the identification of the Part and Chapter provided within the text.
2. Skim the document to understand its overall theme and intent. 
3. Extract the core essence and primary themes of the document, ensuring to represent the author's primary intent accurately.
4. Condense any significant points, arguments, or charatcer's thoughts into a clear and succinct overview.
5. For any events mentioned in the text, provide the characters involved, date, time and place of the event ensuring that the essence of the event is captured succinctly.
6. Present a compact summary that encapsulates:
- Condensed significant points, arguments, or charatcer's thoughts into a clear and succinct overview.
- Any events that take place and their outcome.


Your response should be a succinct paragraph, not exceeding 200 words, encapsulating the key information of the text.
 Avoid including any information outside the scope of the text's summary. ALWAYS give a summary of the given text.

Text to be summarized is enclosed within ``` ``` markers:

```{summary_string_refine}```

CONCISE SUMMARY:""")

In [140]:
# Specify the file path where you want to save the text file
file_path = "refine_2.txt"

# Open the file in write mode and save the string
with open(file_path, "w") as file:
    file.write(refine_summary.content)

print(f"String saved to {file_path}")

String saved to refine_2.txt


Refine method for text summarization with LLMs can pull in more relevant context and may be less lossy than Map Reduce.
- There is some potential dependency on the ordering of the documents. Latest documents they might become more relevant as this method suffers from recency bias.
- Requires many more calls to the LLM than Stuffing, and these calls are not independent

# Map Reduce Attempt 2 (without prompts)

In [158]:
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [159]:
num_docs = len(data_sample)

num_tokens_first_doc = llm.get_num_tokens(data_sample[3].page_content)

print (f"Now we have {num_docs} documents and the fourth one has {num_tokens_first_doc} tokens")

Now we have 38 documents and the fourth one has 393 tokens


In [160]:
summary_chain = load_summarize_chain(llm=llm, chain_type='map_reduce')

In [155]:
output = summary_chain.run(data_sample)

In [157]:
mapr_summary_2 = output
# Save the summary to a text file
mapr_summary_2_file_path = "map_reduce_3.txt"

# Open the file in write mode and save the string
with open(mapr_summary_2_file_path, "w") as file:
    file.write(mapr_summary_2)

print(f"String saved to {mapr_summary_2_file_path}")

String saved to map_reduce_3.txt


# Map Reduce Attempt 3

## Approach:  Best Representation Vectors

In [41]:
text = ""
for page in data_sample:
     text += page.page_content

In [ ]:
num_tokens = llm.get_num_tokens(text)

In [43]:
print (f"These two chapters have {num_tokens} tokens in it")

These two chapters have 14999 tokens in it


In [48]:
# Loaders
from langchain.schema import Document

# Splitters
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Model
from langchain.chat_models import ChatOpenAI

# Embedding Support
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# Summarizer we'll use for Map Reduce
from langchain.chains.summarize import load_summarize_chain

# Data Science
import numpy as np
from sklearn.cluster import KMeans

In [61]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\t"], chunk_size=500, chunk_overlap=20)

docs = text_splitter.create_documents([text])

In [62]:
num_documents = len(docs)

print (f"Now 2 chapters split up into {num_documents} documents")

Now 2 chapters split up into 126 documents


In [63]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))

vectors = embeddings.embed_documents([x.page_content for x in docs])

RateLimitError: Error code: 429 - {'error': {'message': 'Your account is not active, please check your billing details on our website.', 'type': 'billing_not_active', 'param': None, 'code': 'billing_not_active'}}

# Initial Try using Azure OpenAI

**Triggers content filters**

In [37]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="chat",
    model="gpt-35-turbo-16k",
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_type="azure",
    api_version = "2023-03-15-preview"
)

KeyError: 'OPENAI_API_BASE'

In [ ]:
map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details of the mentioned story.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

In [ ]:
map_reduce_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

Refine method for text summarization with LLMs can pull in more relevant context and may be less lossy than Map Reduce. However, it requires many more calls to the LLM than Stuffing, and these calls are not independent, meaning they cannot be parallelized. Additionally, there is some potential dependency on the ordering of the documents. Latest documents they might become more relevant as this method suffers from recency bias.

In [ ]:
map_reduce_outputs = map_reduce_chain({"input_documents": text_chunks})

c:\Users\HAROON TRADERS\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


ValueError: Azure has not provided the response due to a content filter being triggered

In [ ]:
question_prompt_template = """
                  Please provide a summary of the following text.
                  TEXT: {text}
                  SUMMARY:
                  """

question_prompt = PromptTemplate(
    template=question_prompt_template, input_variables=["text"]
)

refine_prompt_template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
              """

refine_prompt = PromptTemplate(
    template=refine_prompt_template, input_variables=["text"]
)

In [ ]:
refine_chain = load_summarize_chain(
    llm,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
)

In [ ]:
refine_outputs = refine_chain({"input_documents": text_chunks})

BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400}}

https://atlassc.net/2023/11/18/turning-off-azure-openai-service-s-content-filters